# Read the Data

In [1]:
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import csv
import chardet as ch

def browse_file():
    Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
    filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
    return filename

def load_data(fcsv,dlt, enc = 'ascii'):
    df = pd.read_csv(fcsv, delimiter = dlt, encoding = enc)
    pd.options.display.float_format = '{:.0f}'.format # Make pandas display in scientific format for all continuous computing
    return df

def csvcomma(fcsv) : # We want to find out the delimiter
    rawdata = open(fcsv).read()
    sniffer = csv.Sniffer()
    dialect = sniffer.sniff(rawdata)
    return dialect.delimiter

def csvencode(fcsv) : # csv encoding for good reading
    rawdata = open(fcsv,'rb').read()
    result = ch.detect(rawdata)
    charenc = result['encoding']
    return charenc

In [2]:
filecsv = browse_file()
delit = csvcomma(filecsv)
encd = csvencode(filecsv)
df = load_data(filecsv, delit, encd)
df.head()

,No,Period,AC,Zone,Monthly
0,1,2016 01,1011000000000,Zone 01,1284294
1,2,2016 02,1011000000000,Zone 01,1639863
2,3,2016 03,1011000000000,Zone 01,1
3,4,2016 04,1011000000000,Zone 01,1995432
4,5,2016 05,1011000000000,Zone 01,5365377


# Overview the Data

In [3]:
def overview(df) :
    print('This is the describe for data continuous :\n')
    print(df.describe())
    print('_'*80+'\n')
    print('This is the describe for data Object :\n')
    print(df.describe(include='O'))
    print('_'*80+'\n')
    print('This is head of data :\n')
    print(df.head(4))
    print('_'*80+'\n')
    print('This is type of data in each columns :\n')
    print(df.dtypes)
    print('_'*80+'\n')    
    print('This is number of NaN in data :\n')
    print(df.isna().sum())

overview(df)

This is the describe for data continuous :

          No            AC    Monthly
count 102239        102239     102239
mean   51120 1011326307378    5724675
std    29514     270408538   61125496
min        1 1011000000000          0
25%    25560 1011010000000          0
50%    51120 1011400000000          1
75%    76680 1011600000000     472468
max   102239 1012000000000 4695129768
________________________________________________________________________________

This is the describe for data Object :

         Period      Zone
count    102239    102239
unique       48        12
top     2019 12  National
freq       2444     15482
________________________________________________________________________________

This is head of data :

   No   Period            AC     Zone  Monthly
0   1  2016 01 1011000000000  Zone 01  1284294
1   2  2016 02 1011000000000  Zone 01  1639863
2   3  2016 03 1011000000000  Zone 01        1
3   4  2016 04 1011000000000  Zone 01  1995432
_____________________

In [4]:
##Customize the Date attribute
from datetime import datetime

def att_time(df, att, dlt) :
    new = df[att].str.split(dlt, n = 1, expand = True) #we split the period attribute into 2 columns by the space
    #display(new.head(3))
    df['Month'] = new[1]
    df['Year'] = new[0]
    #display(df.head(3)) # We got 2 columns with the month and year

    s1 = df['Month'].tolist()
    s2 = df['Year'].tolist()

    s3 = [] #make a container for date parsing
    for i in range(len(s1)) :
        s4 = '{}-{}-01'.format(s2[i], s1[i])
        s4 = datetime.strptime(s4,'%Y-%m-%d').date() #make it type date by parse the date format in string type
        s3.append(s4) #add the result into s3 list
    
    df['Date'] = s3 #make it part of Data attribute
    df.drop([att],axis = 1) #try to remove att attribute in data
    df.set_index('Date',inplace=True) #Set date attribute as the index
    #df.reset_index(inplace=True)
    display(df.head(3))
    
att_time(df, att='Period', dlt=' ')
df.drop(['Period'],axis = 1)
df.rename(columns={'Monthly':'Num_Sales'}, inplace=True)

,No,Period,AC,Zone,Monthly,Month,Year
Date,,,,,,,
2016-01-01,1,2016 01,1011000000000,Zone 01,1284294,01,2016
2016-02-01,2,2016 02,1011000000000,Zone 01,1639863,02,2016
2016-03-01,3,2016 03,1011000000000,Zone 01,1,03,2016
